In [1]:
import sys

IN_COLAB = "google.colab" in sys.modules
IN_COLAB

True

In [2]:
if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
if IN_COLAB:
    # Put here the full path to the folder having your Sprint project code
    # e.g. "/content/drive/MyDrive/assignment"
    ROOT_DIR = "/content/drive/MyDrive/'Colab Notebooks'/AnyoneAI/final_project/main"
    %cd $ROOT_DIR

/content/drive/MyDrive/Colab Notebooks/AnyoneAI/final_project/main


In [4]:
pip install boto3 python_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from src import config, data_utils, evaluation, plot

# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

In [6]:
dataset = pd.read_csv('dataset.csv')

In [7]:
# perform random downsampling
sample_size = dataset['TARGET_LABEL_BAD=1'].value_counts().min()
dataset_resampled = pd.concat([dataset[dataset['TARGET_LABEL_BAD=1'] == c].sample(sample_size) for c in dataset['TARGET_LABEL_BAD=1'].unique()])
print('Resampled dataset shape %s' % dataset_resampled['TARGET_LABEL_BAD=1'].value_counts())

Resampled dataset shape 1    13018
0    13018
Name: TARGET_LABEL_BAD=1, dtype: int64


In [8]:
# first split in sets
app_train_set, app_val_set, app_test_set = data_utils.get_feature_in_set(dataset_resampled)

In [9]:
# second split features and target
train_prep, y_train, val_prep, y_val, test_prep, y_test = data_utils.get_feature_target(app_train_set,app_val_set,app_test_set)

In [10]:
# third apply preprocess in sets
X_train, X_val, X_test, columns = data_utils.preprocess_data(train_prep,val_prep,test_prep)

Input train data shape:  (18745, 17)
Input val data shape:  (2083, 17)
Input test data shape:  (5208, 17) 

cat_cols:  Index(['APPLICATION_SUBMISSION_TYPE', 'SEX', 'RESIDENCIAL_STATE',
       'FLAG_RESIDENCIAL_PHONE', 'COMPANY', 'FLAG_PROFESSIONAL_PHONE',
       'PAYMENT_DAY', 'MARITAL_STATUS', 'NACIONALITY', 'RESIDENCE_TYPE',
       'MONTHLY_INCOMES_TOT', 'FLAG_CARDS', 'QUANT_BANKING_ACCOUNTS_TOT',
       'PERSONAL_ASSETS_VALUE', 'QUANT_CARS'],
      dtype='object')
['QUANT_DEPENDANTS', 'MONTHS_IN_RESIDENCE']
Index(['SEX', 'FLAG_RESIDENCIAL_PHONE', 'COMPANY', 'FLAG_PROFESSIONAL_PHONE',
       'PAYMENT_DAY', 'NACIONALITY', 'FLAG_CARDS',
       'QUANT_BANKING_ACCOUNTS_TOT', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS'],
      dtype='object')
Index(['APPLICATION_SUBMISSION_TYPE', 'RESIDENCIAL_STATE', 'MARITAL_STATUS',
       'RESIDENCE_TYPE', 'MONTHLY_INCOMES_TOT'],
      dtype='object')


In [11]:
lgbm=lgb.LGBMClassifier()
parameters = {'num_leaves':[80,90,95,100], 'min_child_samples':[5,10,15],'max_depth':[-1,5],
             'learning_rate':[0.01,0.05,0.1,0.2]}
baseline_model = RandomizedSearchCV(lgbm,parameters,scoring='accuracy',n_iter=15)

In [12]:
baseline_model.fit(X_train, y_train)

RandomizedSearchCV(estimator=LGBMClassifier(), n_iter=15,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [-1, 5],
                                        'min_child_samples': [5, 10, 15],
                                        'num_leaves': [80, 90, 95, 100]},
                   scoring='accuracy')

In [13]:
y_hat = baseline_model.predict(X_test)

In [14]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58      2629
           1       0.57      0.55      0.56      2579

    accuracy                           0.57      5208
   macro avg       0.57      0.57      0.57      5208
weighted avg       0.57      0.57      0.57      5208



In [15]:
# Best param_num_leaves:90, param_min_child_samples:20, param_max_depth:5, param_learning_rate:0.1

In [16]:
res_logistic = pd.DataFrame(baseline_model.cv_results_)
res_logistic.sort_values(by='rank_test_score', ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_num_leaves,param_min_child_samples,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.177940,0.004501,0.018537,0.002838,80,15,5,0.05,"{'num_leaves': 80, 'min_child_samples': 15, 'm...",0.566018,0.557749,0.567085,0.579354,0.560950,0.566231,0.007387,1
14,0.178934,0.004628,0.018227,0.002745,95,15,5,0.05,"{'num_leaves': 95, 'min_child_samples': 15, 'm...",0.566018,0.557749,0.567085,0.579354,0.560950,0.566231,0.007387,1
12,0.183724,0.008346,0.015993,0.000352,95,10,5,0.05,"{'num_leaves': 95, 'min_child_samples': 10, 'm...",0.564150,0.562550,0.565217,0.576687,0.559883,0.565698,0.005781,3
11,0.177079,0.010047,0.017528,0.003744,80,10,5,0.1,"{'num_leaves': 80, 'min_child_samples': 10, 'm...",0.560950,0.562817,0.571352,0.573753,0.558549,0.565484,0.005976,4
9,0.180617,0.005783,0.016572,0.001347,100,5,5,0.05,"{'num_leaves': 100, 'min_child_samples': 5, 'm...",0.562550,0.562550,0.564951,0.574286,0.560683,0.565004,0.004835,5
4,0.200727,0.009104,0.013455,0.000955,95,10,5,0.01,"{'num_leaves': 95, 'min_child_samples': 10, 'm...",0.559349,0.563083,0.570552,0.563884,0.555081,0.562390,0.005138,6
5,0.746269,0.934871,0.023701,0.009922,95,5,5,0.2,"{'num_leaves': 95, 'min_child_samples': 5, 'ma...",0.559349,0.558282,0.569485,0.565751,0.556682,0.561910,0.004882,7
1,0.813867,0.207292,0.036415,0.013148,100,15,5,0.2,"{'num_leaves': 100, 'min_child_samples': 15, '...",0.555348,0.562550,0.569752,0.568418,0.553214,0.561856,0.006677,8
6,0.166036,0.005457,0.016464,0.003116,95,15,5,0.2,"{'num_leaves': 95, 'min_child_samples': 15, 'm...",0.555348,0.562550,0.569752,0.568418,0.553214,0.561856,0.006677,8
8,0.200925,0.009201,0.013197,0.000497,90,15,5,0.01,"{'num_leaves': 90, 'min_child_samples': 15, 'm...",0.557482,0.564150,0.570285,0.562283,0.554281,0.561696,0.005530,10
